In [ ]:










import os
# import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Directory paths to your audio files
gunshot_dir = 'gunshot'
non_gunshot_dir = 'not_gunshot'

# Function to create the dataset (labels and file paths)
def create_dataset(gunshot_dir, non_gunshot_dir):
    file_paths = []
    labels = []

    # Add gunshot files and label them as 1
    for filename in os.listdir(gunshot_dir):
        if filename.endswith('.wav') or filename.endswith('.mp3'):
            file_paths.append(os.path.join(gunshot_dir, filename))
            labels.append('gunshot')
    
    # Add non-gunshot files and label them as 0
    for filename in os.listdir(non_gunshot_dir):
        if filename.endswith('.wav') or filename.endswith('.mp3'):
            file_paths.append(os.path.join(non_gunshot_dir, filename))
            labels.append('non-gunshot')

    return pd.DataFrame({'file_name': file_paths, 'label': labels})

# Create a dataframe with file paths and labels
df = create_dataset(gunshot_dir, non_gunshot_dir)

# Show the first few rows of the dataset
print(df.head())
df

ModuleNotFoundError: No module named 'librosa'

In [ ]:
import librosa
import numpy as np

# Define a fixed length for MFCCs
MAX_MFCC_LENGTH = 100  # Adjust as needed

    # Pad or truncate MFCCs to MAX_MFCC_
def extract_features(file_path, max_len=100):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Shape: (13, time_steps)

    # Transpose to (time_steps, 13)
    mfcc = mfcc.T

    # Fix length (pad or truncate)
    if mfcc.shape[0] < max_len:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:max_len, :]  # Ensure shape is (max_len, 13)

    return mfcc


# Extract features for all audio files
X = np.array([extract_features(fp) for fp in df['file_name'].values], dtype=np.float32)

# Encode labels (Gunshot/Non-Gunshot)
encoder = LabelEncoder()
y = encoder.fit_transform(df['label'].values)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Preprocessing Completed.")


Data Preprocessing Completed.


In [ ]:
X = np.array([extract_features(fp) for fp in df['file_name'].values])

# Print shape to debug
print(f"Shape of X: {X.shape}")  # Expected (num_samples, 100, 13)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")  # Expected (num_samples, 100, 13)


Shape of X: (456, 100, 13)
X_train shape: (364, 100, 13)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import layers

model = Sequential()
model.add(LSTM(64, input_shape=(100, 13), return_sequences=True))  # Use correct input shape
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

def pad_or_truncate(mfcc, max_len=100):
    if mfcc.shape[0] < max_len:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:max_len, :]
    return mfcc

X_train = np.array([pad_or_truncate(x) for x in X_train])
X_test = np.array([pad_or_truncate(x) for x in X_test])

# Reshape input data for LSTM (3D shape: samples, timesteps, features)
#X_train = np.array([librosa.util.fix_length(x.T, size=100).T for x in X_train])
#X_test = np.array([librosa.util.fix_length(x.T, size=100).T for x in X_test])


# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save('gunshot_detection_model03.h5')

print("Model Training Completed.")


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.6926 - loss: 0.5997 - val_accuracy: 0.8804 - val_loss: 0.3547
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.9484 - loss: 0.2298 - val_accuracy: 0.8804 - val_loss: 0.3231
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9485 - loss: 0.1626 - val_accuracy: 0.8804 - val_loss: 0.3360
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9515 - loss: 0.1454 - val_accuracy: 0.8804 - val_loss: 0.2752
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9576 - loss: 0.0971 - val_accuracy: 0.8804 - val_loss: 0.2338
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9568 - loss: 0.0704 - val_accuracy: 0.8913 - val_loss: 0.2179
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.9678 - loss: 0.0623 - val_accuracy: 0.9130 - val_loss: 0.2046
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.9797 - loss: 0.0463 - val_accuracy: 0

Model Training Completed.


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9587 - loss: 0.2958
Test Accuracy: 95.65%


In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("gunshot_detection_model03.h5")


In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("gunshot_detection_model03.h5")

# Create a converter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable TF Select Ops for unsupported operations (like LSTM)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Allow standard TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Allow TF ops not natively supported in TFLite
]

# Disable experimental lowering of tensor list ops (Fixes TensorListReserve error)
converter._experimental_lower_tensor_list_ops = False

# Convert the model
tflite_model = converter.convert()

# Save the converted TFLite model
with open("gunshot_detection_model03.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite!")


INFO:tensorflow:Assets written to: /tmp/tmp09ng9n_l/assets


INFO:tensorflow:Assets written to: /tmp/tmp09ng9n_l/assets


Saved artifact at '/tmp/tmp09ng9n_l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 13), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137608937316304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937319760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937319184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937321104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937322064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937322640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937322256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137608937323408: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model successfully converted to TensorFlow Lite!


W0000 00:00:1743045192.008316    5404 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743045192.008358    5404 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-27 08:43:12.008932: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp09ng9n_l
2025-03-27 08:43:12.010722: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-27 08:43:12.010743: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp09ng9n_l
I0000 00:00:1743045192.028503    5404 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-03-27 08:43:12.031173: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-27 08:43:12.115804: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp09ng9n_l
2025-03-27 08:43:12.145175: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("gunshot_detection_model03.h5")

# Create a converter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable TF Select Ops for unsupported operations (like LSTM)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Allow standard TFLite ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Allow TF ops not natively supported in TFLite
]

# Disable experimental lowering of tensor list ops (Fixes TensorListReserve error)
converter._experimental_lower_tensor_list_ops = False

# Convert the model
tflite_model = converter.convert()

# Save the converted TFLite model
with open("gunshot_detection_model03.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite!")


In [ ]:
import numpy as np
import librosa
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="gunshot_detection_model03.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess audio (same as during training)
def extract_features(file_path, max_pad_len=100):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)  # Use 13 MFCCs instead of 40
    
    # Pad/Crop to fixed length
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(pad_width, 0))))
    mfccs = mfccs[:, :max_pad_len]  # Crop if longer
    
    # Reshape to (1, 100, 13) - transpose and add batch dim
    mfccs = mfccs.T  # Transpose to (100, 13)
    mfccs = np.expand_dims(mfccs, axis=0)  # Add batch dim: (1, 100, 13)
    
    return mfccs.astype(np.float32) # Add batch dim: (1, 100, 40)
    
    return mfccs.astype(np.float32)
# Example: Predict on a new audio file
def predict_gunshot(audio_path, threshold=0.5):
    features = extract_features(audio_path)  # Now shape (1, 100, 13)
    
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], features)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])
    
    is_gunshot = prediction[0][0] > threshold
    confidence = prediction[0][0] if is_gunshot else 1 - prediction[0][0]
    label = "NOT-GUNSHOT" if is_gunshot else " GUNSHOT"
    
    print(f"Prediction: {label} (Confidence: {confidence:.4f})")
    return is_gunshot, confidence
predict_gunshot("ak-47-14501.mp3")  # Replace with your audio file

Prediction:  GUNSHOT (Confidence: 0.9999)


(False, 0.9999304464581655)